In [12]:
import numpy as np
import pandas as pd
from matplotlib import style
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from numpy import array
from sklearn.metrics import accuracy_score
from numpy import argmax
from sklearn.preprocessing import StandardScaler

import os
import gc


In [13]:
import math
def normalize(x, xmin, xmax, a, b):
    numerator = x - xmin
    denominator = xmax - xmin
    multiplier = b - a
    ans = (numerator/denominator)*multiplier + a
    return ans

sig_min = -100
sig_max = 0
tar_min = 0.25
tar_max = 1.0
no_sig = 100

def normalize_wifi(num):
    ans = 0
    num = float(num)
    if math.isclose(num, no_sig, rel_tol=1e-3):
        return 0
    else:
        ans = normalize(num, sig_min, sig_max, tar_min, tar_max)
        return ans

In [14]:
#SODIndoor dataset SYL building

df_train=pd.read_csv('data/Training_SYL.csv')
df_test=pd.read_csv('data/Testing_SYL.csv')
df_train.head()

,MAC1,MAC2,MAC3,MAC4,MAC5,MAC6,MAC7,MAC8,MAC9,MAC10,...,MAC362,MAC363,ECoord,NCoord,FloorID,BuildingID,SceneID,UserID,PhoneID,SampleTimes
0,-74,-65,-57,-67,-79,-49,-75,-49,-73,-82,...,100,100,19.03,20.05,4,3,1,1,1,1
1,-74,-65,-57,-67,-79,-49,-75,-49,-73,-82,...,100,100,19.03,20.05,4,3,1,1,1,2
2,-74,-65,-57,-67,-79,-49,-75,-49,-73,-82,...,100,100,19.03,20.05,4,3,1,1,1,3
3,-74,-65,-57,-67,-79,-49,-75,-49,-73,-82,...,100,100,19.03,20.05,4,3,1,1,1,4
4,-74,-65,-57,-67,-79,-49,-75,-49,-73,-82,...,100,100,19.03,20.05,4,3,1,1,1,5


In [15]:
#taking 2.4 GHz Macs and 5GHz macs
#2.4 List2, 5 GHz list 5
list2=['MAC1', 'MAC47', 'MAC21', 'MAC17', 'MAC5', 'MAC35', 'MAC3', 'MAC27', 'MAC49', 'MAC31', 'MAC51', 'MAC29', 'MAC23', 'MAC39', 'MAC41', 'MAC19', 'MAC33', 'MAC43', 'MAC37', 'MAC9', 'MAC45', 'MAC25', 'MAC7', 'MAC13', 'MAC15', 'MAC11']
list5=['MAC2', 'MAC48', 'MAC22', 'MAC18', 'MAC6', 'MAC36', 'MAC4', 'MAC28', 'MAC50', 'MAC32', 'MAC52', 'MAC30', 'MAC24', 'MAC40', 'MAC42', 'MAC20', 'MAC34', 'MAC44', 'MAC38', 'MAC10', 'MAC46', 'MAC26', 'MAC8', 'MAC14', 'MAC16', 'MAC12']


label_cols=['ECoord', 'NCoord', 'FloorID']
common_feats= list(set(df_train.columns).intersection(set(list2))) + list(set(df_train.columns).intersection(set(list5))) + label_cols


df_train=df_train[common_feats]
df_test=df_test[common_feats]

#drop duplicates
df_train = df_train.drop_duplicates(keep='first')
df_train=df_train.reset_index(drop=True)

df_test = df_test.drop_duplicates(keep='first')
df_test=df_test.reset_index(drop=True)


df_train['datatype']=1
df_test['datatype']=2

#merging df_train and df_test for making a csv
frames = [df_train, df_test]
df= pd.concat(frames)
df=df.reset_index(drop=True)

# normalizing signals
col_num=df.shape[1]-4
wifi_cells_train = df.columns[0:col_num]

for i in wifi_cells_train:
    df[i] = df[i].apply(normalize_wifi)
  
    
#df.to_csv("data/SOD_SYL_dual_merged_normalized_dropped.csv",index=False)

print(df_train.shape)
print(df_test.shape)
print(df.shape)

(723, 56)
(139, 56)
(862, 56)


In [16]:
df.head()

,MAC17,MAC3,MAC5,MAC41,MAC15,MAC45,MAC9,MAC21,MAC51,MAC1,...,MAC8,MAC38,MAC22,MAC48,MAC14,MAC28,ECoord,NCoord,FloorID,datatype
0,0.0000,0.5725,0.4075,0.0,0.415,0.43,0.4525,0.5050,0.0,0.4450,...,0.6325,0.4450,0.5875,0.000,0.000,0.355,19.03,20.05,4,1
1,0.0000,0.5725,0.4075,0.0,0.415,0.43,0.4525,0.5050,0.0,0.4450,...,0.6325,0.4450,0.5875,0.000,0.000,0.355,19.03,20.05,4,1
2,0.0000,0.5725,0.4075,0.0,0.415,0.43,0.4525,0.5050,0.0,0.4450,...,0.6325,0.4450,0.5875,0.000,0.000,0.355,19.03,20.05,4,1
3,0.4975,0.5500,0.0000,0.0,0.000,0.34,0.4525,0.5125,0.0,0.3925,...,0.7225,0.4375,0.6325,0.475,0.355,0.000,19.03,18.85,4,1
4,0.4975,0.5500,0.3775,0.0,0.000,0.34,0.4525,0.5125,0.0,0.3925,...,0.7225,0.4375,0.6325,0.475,0.355,0.000,20.23,20.05,4,1
